In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time

In [2]:
driver = webdriver.Chrome(executable_path=r'C:\Program Files\Google\Chrome\Application\chromedriver.exe')

In [3]:
driver.get("https://www.linkedin.com/jobs/search/")

In [4]:
driver.find_element_by_xpath("/html/body/div[1]/header/nav/div/a[2]").click() 

In [5]:
username = driver.find_element_by_id("username")
username.send_keys("your mail") 
pword = driver.find_element_by_id("password")
pword.send_keys("Your Password") 

In [6]:
driver.find_element_by_xpath("//button[@type='submit']").click()
time.sleep(5)

In [7]:
def get_job_card_list(vertical):
    search_word = driver.find_element_by_xpath("/html/body/div[5]/header/div/div/div/div[2]/div[1]/div/div/input[1]")
    time.sleep(5)
    search_word.send_keys(vertical)
    time.sleep(5)
    driver.find_element_by_xpath("/html/body/div[5]/header/div/div/div/div[2]/button[1]").click()
    time.sleep(5)
    #We find how many jobs are offered.
    jobs_num = driver.find_element(By.CSS_SELECTOR,"#main > div > section.scaffold-layout__list > header > div.jobs-search-results-list__title-heading > small").get_attribute("innerText")
    if len(jobs_num.split(',')) > 1:
        jobs_num = int(jobs_num.split(',')[0])*1000
    else:
        jobs_num = int(jobs_num)
    current_url = driver.current_url
    
    return ([jobs_num,current_url])

In [8]:
def get_url_page_list(start_url):
    url = []
    count = 0
    base_url = start_url
    for j in range (22):
        #current_page_element = driver.find_elements_by_xpath('/html/body/div[5]/div[3]/div[4]/div/div/main/div/section[1]/div/div[6]/ul/li[2]/button')
        current_page = str(base_url)+"&start="+str(count)
        url.append(current_page)
        count = count+25
    print(url)
    print(len(url))
    return (url)
#current_page_element = driver.find_elements_by_xpath('//*[@class="artdeco-pagination__indicator artdeco-pagination__indicator--number active selected ember-view"]')

In [9]:
#We declare void list to keep track of all obtaind data.
job_title_list = []
company_name_list = []
location_list = []
date_list= []
job_desc_list = [] 
skills_list = []
data= {}

In [10]:
def scrape_jobs(url):
    for page in url:
        print("Fetching details for the page %s"%page)
        driver.get(page)
        time.sleep(20)
        driver.implicitly_wait(30)
        jobs = driver.find_elements_by_css_selector("ul.scaffold-layout__list-container > li")
    
        for job in jobs:
            job.click()
            time.sleep(10)
            #content = BeautifulSoup(driver.page_source, 'html5lib')
            driver.implicitly_wait(30)
            job_title = driver.find_elements_by_css_selector('div.jobs-unified-top-card__content--two-pane > a > h2')[0].get_attribute("innerText")
            job_title_list.append(job_title )
            driver.implicitly_wait(10)
            company_name = driver.find_elements_by_css_selector('div.jobs-unified-top-card__primary-description > span > span')[0].get_attribute("innerText")
            company_name_list.append(company_name)
            driver.implicitly_wait(10)
            location = driver.find_elements_by_css_selector('#main > div > section.scaffold-layout__detail.overflow-x-hidden.jobs-search__job-details > div > div.job-view-layout.jobs-details > div:nth-child(1) > div > div:nth-child(1) > div > div.relative > div.jobs-unified-top-card__content--two-pane > div.jobs-unified-top-card__primary-description > span.jobs-unified-top-card__subtitle-primary-grouping.t-black > span.jobs-unified-top-card__bullet')[0].get_attribute("innerText")
            location_list.append(location)
            driver.implicitly_wait(10)
            full_JD = driver.find_elements_by_css_selector('#job-details')[0].get_attribute("innerText")
            job_desc_list.append(full_JD)
            driver.implicitly_wait(10)
            skills=[]
            try:
                driver.find_elements_by_css_selector('#how-you-match-card > section:nth-child(2) > div > button > li-icon ')[0].click()
            #driver.find_elements_by_css_selector('#how-you-match-card > section:nth-child(2) > div > button > li-icon ')[0].click()
                driver.implicitly_wait(10)
                time.sleep(10)
                content = BeautifulSoup(driver.page_source, 'html5lib')
                skills_content=content.select('ul.job-details-skill-match-status-list>li')
                for i in skills_content:
                    skills.append(i.select('div>div')[1].get_text().strip())   
                skills_list.append(skills)
                driver.find_elements_by_css_selector(' div > div > button > li-icon')[0].click()
            except:
                skills_list.append([])
        data = {
            "job_title":job_title_list ,
             "company":company_name_list,
             "location":location_list,
             "job_desc":job_desc_list,
             "skills":skills_list
                    }
    return (data)

In [18]:
vertical_list = ["Web Development",
"Networking",
"Hardware ASIC Design",
"Devops",
"Cloud Computing",
"Data Science",
"Project Management",
"Embedded Systems",
"IT Support"]

for vertical in vertical_list:
    print(vertical)
    result = get_job_card_list("Devops")
    print("total_jobs:%s" %result[0])
    print("starturl:%s" %result[1])
    start_page = result[1]
    All_pages=get_url_page_list(str(start_page))
    print(All_pages)
    time.sleep(50)
    print("sleep for 50 sec")
    data=scrape_jobs(All_pages)
    print(data)
    print(len(job_title_list))
    print(len(company_name_list))
    print(len(location_list))
    print(len(job_desc_list))
    print(len(skills_list))
    df = pd.DataFrame(data)
    filename = vertical +".csv"
    df.to_csv(filename)

Fetching details for the page https://www.linkedin.com/jobs/search/?currentJobId=3498342351&geoId=102713980&keywords=Devops&location=India&refresh=true&start=175
Fetching details for the page https://www.linkedin.com/jobs/search/?currentJobId=3498342351&geoId=102713980&keywords=Devops&location=India&refresh=true&start=200
Fetching details for the page https://www.linkedin.com/jobs/search/?currentJobId=3498342351&geoId=102713980&keywords=Devops&location=India&refresh=true&start=225
Fetching details for the page https://www.linkedin.com/jobs/search/?currentJobId=3498342351&geoId=102713980&keywords=Devops&location=India&refresh=true&start=250
Fetching details for the page https://www.linkedin.com/jobs/search/?currentJobId=3498342351&geoId=102713980&keywords=Devops&location=India&refresh=true&start=275
Fetching details for the page https://www.linkedin.com/jobs/search/?currentJobId=3498342351&geoId=102713980&keywords=Devops&location=India&refresh=true&start=300
Fetching details for the pag